# RATIO 2019 - Benchmarking Workshop

In [1]:
import pandas as pd
import numpy as np
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
data_cross_path = 'data/cross-topic/{}.csv'
data_within_path = 'data/within-topic/{}.csv'

In [3]:
import keras.backend as K
from keras.models import Sequential, Model, clone_model, load_model
from keras.callbacks import Callback
from keras.optimizers import *
from keras.layers import *
from keras import *
from keras.utils import to_categorical, Sequence
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform

Using TensorFlow backend.


In [4]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [5]:
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score, f1_score
def report_training_results(y_test, y_pred):
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))  
    print()
    print('Accuracy: ', round(accuracy_score(y_test, y_pred), 2))  #
    print()

    print('Report:')
    print(classification_report(y_test, y_pred))  
    f1_dic = {}
    
    f1_dic['macro'] = round(f1_score(y_pred=y_pred, y_true=y_test, average='macro'), 2)
    f1_dic['micro'] = round(f1_score(y_pred=y_pred, y_true=y_test, average='micro'), 2)
    return f1_dic

### Load within-topics and cross-topics data

In [6]:
cross_train_df = pd.read_csv(data_cross_path.format('train_rand'), index_col='id', escapechar='\\')
# cross_dev_df = pd.read_csv(data_cross_path.format('dev_rand'), index_col='id', escapechar='\\')
# cross_test_df =  pd.read_csv(data_cross_path.format('test_rand'), index_col='id', escapechar='\\')

within_train_df =  pd.read_csv(data_within_path.format('train_rand'), index_col='id', escapechar='\\')
within_dev_df =  pd.read_csv(data_within_path.format('dev_rand'), index_col='id', escapechar='\\')
within_test_df =  pd.read_csv(data_within_path.format('test_rand'), index_col='id', escapechar='\\')

In [7]:
# Adding a tag for the topics in focus: "gay marriage" and "abortion"
def add_tag(row):
    title = row['topic'].lower().strip()
    if title.find('abortion') > -1 :
        row['tag'] = 'abortion'
    elif title.find('gay marriage') > -1 :
        row['tag'] = 'gay marriage'
    else:
        row['tag'] = 'NA'
    return row

cross_train_df = cross_train_df.apply(add_tag, axis=1)
cross_dev_df = cross_dev_df.apply(add_tag, axis=1)
cross_test_df = cross_test_df.apply(add_tag, axis=1)

within_train_df = within_train_df.apply(add_tag, axis=1)
within_dev_df = within_dev_df.apply(add_tag, axis=1)
within_test_df = within_test_df.apply(add_tag, axis=1)

In [8]:
cross_train_df.head(10)

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
0,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
1,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
2,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
3,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00002-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00002-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
4,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
5,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
6,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
7,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
8,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00002-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00002-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion


In [9]:
cross_test_df.head(10)

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
28,"cakerman, thank you for your reply. i will beg...",113c0181-2019-04-18T11:51:11Z-00007-000,"i am pro life, and i believe abortion on deman...",113c0181-2019-04-18T11:51:11Z-00007-000,113c0181-2019-04-18T11:51:11Z,True,abortion on demand should be illegal,abortion
73,those rescued from abortion should be killed i...,11d51208-2019-04-18T15:14:47Z-00000-000,those rescued from abortion should be killed i...,11d51208-2019-04-18T15:14:47Z-00000-000,11d51208-2019-04-18T15:14:47Z,False,those rescued from abortion should be killed i...,abortion
106,thank-you for accepting this debate. hopefully...,11ed3edf-2019-04-18T16:03:18Z-00005-000,abortion is morally permissible as a fetus doe...,11ed3edf-2019-04-18T16:03:18Z-00005-000,11ed3edf-2019-04-18T16:03:18Z,False,abortion (is it ok) part ii,abortion
177,"single issue voting on abortion, for president...",1463f7db-2019-04-18T20:00:30Z-00000-000,"single issue voting on abortion, for president...",1463f7db-2019-04-18T20:00:30Z-00000-000,1463f7db-2019-04-18T20:00:30Z,False,"single issue voting on abortion, for president...",abortion
218,prochoice folks should agree: babies born duri...,1557725f-2019-04-18T16:09:03Z-00000-000,prochoice folks should agree: babies born duri...,1557725f-2019-04-18T16:09:03Z-00000-000,1557725f-2019-04-18T16:09:03Z,True,prochoice folks should agree: babies born duri...,abortion
252,observations obv 1 - the bop is shared in this...,157796c-2019-04-18T13:44:25Z-00004-000,"thank you, famousdebater, for accepting this d...",157796c-2019-04-18T13:44:25Z-00004-000,157796c-2019-04-18T13:44:25Z,False,the usfg should ban abortion.,abortion
267,[--right to abortion--] my opponent has failed...,158fb830-2019-04-18T18:13:11Z-00002-000,you're argument uses lots of high falutin lang...,158fb830-2019-04-18T18:13:11Z-00002-000,158fb830-2019-04-18T18:13:11Z,False,abortion is moraly necessary,abortion
326,abortion is wrong and therefore should be ille...,16683ec1-2019-04-18T19:28:06Z-00003-000,abortion is wrong and therefore should be ille...,16683ec1-2019-04-18T19:28:06Z-00003-000,16683ec1-2019-04-18T19:28:06Z,False,abortion is wrong and therefore should be ille...,abortion
327,abortion is wrong and therefore should be ille...,16683ec1-2019-04-18T19:28:06Z-00003-000,abortion is wrong and therefore should be ille...,16683ec1-2019-04-18T19:28:06Z-00003-000,16683ec1-2019-04-18T19:28:06Z,False,abortion is wrong and therefore should be ille...,abortion


In [10]:
within_train_df.head(10)

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
2607,accepted. pro may extend their arguments to th...,2a0d32eb-2019-04-18T11:46:44Z-00004-000,"i""m pro-life. just think about it, your murder...",2a0d32eb-2019-04-18T11:46:44Z-00004-000,2a0d32eb-2019-04-18T11:46:44Z,False,abortion (pro life),abortion
14632,ultrasounds fit well with pro-choice concepts.,475596d3-2019-04-17T11:47:21Z-00031-000,ultrasounds are a procedure any pregnant woman...,475596d3-2019-04-17T11:47:21Z-00031-000,475596d3-2019-04-17T11:47:21Z,True,mandatory ultrasounds before abortions,abortion
84168,marriage is defined as between a man and woman,d2f4b1cd-2019-04-17T11:47:27Z-00092-000,marriage is celebrated because of the assumpti...,d2f4b1cd-2019-04-17T11:47:27Z-00092-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage
58293,"there is no inviolable ""right to life"" in abor...",b67fc3fb-2019-04-17T11:47:41Z-00102-000,"parents must ""control their bodies"" or else ri...",b67fc3fb-2019-04-17T11:47:41Z-00102-000,b67fc3fb-2019-04-17T11:47:41Z,False,abortion,abortion
85569,"al rantell, a homosexual talk-show host in la....",d2f4b1cd-2019-04-17T11:47:27Z-00154-000,denying marriage to infertile would be too costly,d2f4b1cd-2019-04-17T11:47:27Z-00154-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage
51234,the reproductive rights movement has no genoci...,b67fc3fb-2019-04-17T11:47:41Z-00024-000,legal abortion protects women with serious ill...,b67fc3fb-2019-04-17T11:47:41Z-00024-000,b67fc3fb-2019-04-17T11:47:41Z,True,abortion,abortion
16228,a woman has the right to her own body. not nea...,47ca97e8-2019-04-18T18:54:04Z-00004-000,abortion should be illegal in the united states.,47ca97e8-2019-04-18T18:54:04Z-00004-000,47ca97e8-2019-04-18T18:54:04Z,False,abortion,abortion
41532,abortion is important to reducing over-populat...,b67fc3fb-2019-04-17T11:47:41Z-00012-000,"life is an individual right, not a privilege, ...",b67fc3fb-2019-04-17T11:47:41Z-00012-000,b67fc3fb-2019-04-17T11:47:41Z,True,abortion,abortion
89518,children have a claim to biological parents,d2f4b1cd-2019-04-17T11:47:27Z-00028-000,gay spouses can helpfully adopt orphaned kids....,d2f4b1cd-2019-04-17T11:47:27Z-00028-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage


In [11]:
within_test_df.head(10)

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
85249,"gay marriage devalues marriage, frequency of o...",d2f4b1cd-2019-04-17T11:47:27Z-00063-000,being unaccustomed to gay marriage is no argument,d2f4b1cd-2019-04-17T11:47:27Z-00063-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage
40696,fewer women would have abortions if they knew ...,b67fc3fb-2019-04-17T11:47:41Z-00192-000,poor women are disproportionately deprived cho...,b67fc3fb-2019-04-17T11:47:41Z-00192-000,b67fc3fb-2019-04-17T11:47:41Z,True,abortion,abortion
53484,"wanted fetuses are beloved ""babies""; unwanted ...",b67fc3fb-2019-04-17T11:47:41Z-00149-000,it is wrong to consider abortion a tool in cri...,b67fc3fb-2019-04-17T11:47:41Z-00149-000,b67fc3fb-2019-04-17T11:47:41Z,True,abortion,abortion
33657,con is concerned that i have not refuted his p...,b1862425-2019-04-18T15:07:45Z-00002-000,well i am not going to forfeit any rounds. i d...,b1862425-2019-04-18T15:07:45Z-00002-000,b1862425-2019-04-18T15:07:45Z,False,abortion,abortion
9667,civil unions give gays equal benefits w/o chan...,40f91664-2019-04-17T11:47:29Z-00051-000,gay marriage is a negligible change to institu...,40f91664-2019-04-17T11:47:29Z-00051-000,40f91664-2019-04-17T11:47:29Z,False,civil unions vs. gay marriage,gay marriage
92271,"killing baby girls is wrong (& by extension, s...",e6dd9be0-2019-04-18T12:08:18Z-00002-000,"killing baby girls is wrong (& by extension, s...",e6dd9be0-2019-04-18T12:08:18Z-00002-000,e6dd9be0-2019-04-18T12:08:18Z,False,"killing baby girls is wrong (& by extension, s...",abortion
52974,abortion allows women to have sex comfortably ...,b67fc3fb-2019-04-17T11:47:41Z-00178-000,"no woman ""wants"" an abortion; it is only the l...",b67fc3fb-2019-04-17T11:47:41Z-00178-000,b67fc3fb-2019-04-17T11:47:41Z,True,abortion,abortion
26844,countries undergoing demographic transition sh...,8939e4ca-2019-04-18T18:12:15Z-00008-000,countries undergoing demographic transition sh...,8939e4ca-2019-04-18T18:12:15Z-00008-000,8939e4ca-2019-04-18T18:12:15Z,True,countries undergoing demographic transition sh...,abortion
7907,civil partners can only file taxes in register...,40f91664-2019-04-17T11:47:29Z-00069-000,marriage should be defined as between a man an...,40f91664-2019-04-17T11:47:29Z-00069-000,40f91664-2019-04-17T11:47:29Z,True,civil unions vs. gay marriage,gay marriage


# Within topic

In [12]:
# BERTify training and test data
from bert_serving.client import BertClient
bc = BertClient()

KeyboardInterrupt: 

In [ ]:
a1 = bc.encode(within_train_df.argument1.tolist())
a2 = bc.encode(within_train_df.argument2.tolist())
train_embedded_pairs = zip(a1, a2)
pickle.dump(train_embedded_pairs, open("BERT_pairs_train.pkl", "wb"))

In [ ]:
a1 = bc.encode(within_dev_df.argument1.tolist())
a2 = bc.encode(within_dev_df.argument2.tolist())
dev_embedded_pairs = zip(a1, a2)
pickle.dump(dev_embedded_pairs, open("BERT_pairs_dev.pkl", "wb"))

In [ ]:
a1 = bc.encode(within_test_df.argument1.tolist())
a2 = bc.encode(within_test_df.argument2.tolist())
test_embedded_pairs = zip(a1, a2)
pickle.dump(test_embedded_pairs, open("BERT_pairs_test.pkl", "wb"))

## Model

In [13]:
training_embs = pickle.load(open("BERT_pairs_train.pkl","rb"))
dev_embs = pickle.load(open("BERT_pairs_dev.pkl","rb"))
test_embs = pickle.load(open("BERT_pairs_test.pkl","rb"))

In [14]:
t1, t2 = zip(*training_embs)
train_args1 = np.array(t1)
train_args2 = np.array(t2)
print(train_args1.shape)
train_tags = to_categorical([0 if t=='abortion' else 1 for t in within_train_df.tag.tolist()], num_classes=2)

(57467, 1024)


In [15]:
t1, t2 = zip(*dev_embs)
dev_args1 = np.array(t1)
dev_args2 = np.array(t2)
dev_tags = to_categorical([0 if t=='abortion' else 1 for t in within_dev_df.tag.tolist()], num_classes=2)
dev_args1.shape

(4498, 1024)

In [16]:
t1, t2 = zip(*test_embs)
test_args1 = np.array(t1)
test_args2 = np.array(t2)
test_tags = to_categorical([0 if t=='abortion' else 1 for t in within_test_df.tag.tolist()], num_classes=2)
test_args1.shape

(1938, 1024)

In [17]:
train_output = [1 if t else 0 for t in within_train_df.is_same_side]
dev_output = [1 if t else 0 for t in within_dev_df.is_same_side]
test_output = [1 if t else 0 for t in within_test_df.is_same_side]

In [18]:
embedding_dims = t1[0].shape

layer_input_1 = Input(shape = embedding_dims, name = 'input_1')
layer_input_2 = Input(shape = embedding_dims, name = 'input_2')
layer_input_3 = Input(shape = (2,), name = 'input_3')

encoder_1 = Dense(embedding_dims[0], activation='relu')(layer_input_1)
encoder_2 = Dense(embedding_dims[0], activation='relu')(layer_input_2)

# combined_diff = subtract([encoder_1, encoder_2])
# combined_mult = multiply([encoder_1, encoder_2])
combined_dot = dot([encoder_1, encoder_2], axes=-1, normalize=False)
combined_all = concatenate([encoder_1, encoder_2, combined_dot, layer_input_3])

features_pred = Dense(300, activation='relu')(combined_all)
# features_norm = BatchNormalization()(features_pred)
# features_acti = LeakyReLU()(features_pred)
# features_drop = Dropout(rate=0.5)(features_acti)
layer_prediction = Dense(1, activation='sigmoid')(features_pred)

model = Model([layer_input_1, layer_input_2, layer_input_3], layer_prediction)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("argmining19_simple-nn-model.hdf5", monitor='val_acc', 
                             verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [20]:
model.fit([train_args1, train_args2, train_tags], train_output, batch_size=32, epochs=50, 
          validation_data=([dev_args1, dev_args2, dev_tags], dev_output), verbose=True,
          callbacks=[checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 57467 samples, validate on 4498 samples
Epoch 1/50
57467/57467 [==============================] - 23s 404us/step - loss: 0.3694 - acc: 0.7852 - val_loss: 0.2862 - val_acc: 0.8195

Epoch 00001: val_acc improved from -inf to 0.81948, saving model to argmining19_simple-nn-model.hdf5
Epoch 2/50
57467/57467 [==============================] - 13s 224us/step - loss: 0.2809 - acc: 0.8288 - val_loss: 0.2739 - val_acc: 0.8297

Epoch 00002: val_acc improved from 0.81948 to 0.82970, saving model to argmining19_simple-nn-model.hdf5
Epoch 3/50
57467/57467 [==============================] - 13s 224us/step - loss: 0.2654 - acc: 0.8411 - val_loss: 0.2792 - val_acc: 0.8297

Epoch 00003: val_acc did not improve from 0.82970
Epoch 4/50
57467/57467 [==============================] - 13s 224us/step - loss: 0.2526 - acc: 0.8506 - val_loss: 0.2619 - val_acc: 0.8413

Epoch 00004: val_acc improved from 0.82970 to 0.84126, saving model to argmining19_simpl

57467/57467 [==============================] - 12s 216us/step - loss: 0.0748 - acc: 0.9656 - val_loss: 0.5577 - val_acc: 0.8662

Epoch 00038: val_acc improved from 0.86572 to 0.86616, saving model to argmining19_simple-nn-model.hdf5
Epoch 39/50
57467/57467 [==============================] - 12s 216us/step - loss: 0.0773 - acc: 0.9663 - val_loss: 0.5868 - val_acc: 0.8606

Epoch 00039: val_acc did not improve from 0.86616
Epoch 40/50
57467/57467 [==============================] - 12s 216us/step - loss: 0.0791 - acc: 0.9671 - val_loss: 0.6109 - val_acc: 0.8575

Epoch 00040: val_acc did not improve from 0.86616
Epoch 41/50
57467/57467 [==============================] - 12s 215us/step - loss: 0.0661 - acc: 0.9700 - val_loss: 0.6506 - val_acc: 0.8613

Epoch 00041: val_acc did not improve from 0.86616
Epoch 42/50
57467/57467 [==============================] - 12s 216us/step - loss: 0.0666 - acc: 0.9701 - val_loss: 0.6603 - val_acc: 0.8628

Epoch 00042: val_acc did not improve from 0.86616
Epo

In [21]:
model.load_weights("argmining19_simple-nn-model.hdf5")
dev_output_pred = np.round(model.predict([dev_args1, dev_args2, dev_tags]))
test_output_pred = np.round(model.predict([test_args1, test_args2, test_tags]))

In [22]:
# dev set performances
# --------------------

# concatenate([encoder_1, encoder_2, combined_diff, combined_dot])
# Epoch 17/20
# 57512/57512 [==============================] - 12s 207us/step - loss: 0.1906 - acc: 0.8963 - val_loss: 0.3081 - val_acc: 0.8512

# combined_all = concatenate([encoder_1, encoder_2])
# Epoch 17/20
# 57512/57512 [==============================] - 11s 191us/step - loss: 0.1767 - acc: 0.9052 - val_loss: 0.3394 - val_acc: 0.8514

# combined_all = concatenate([encoder_1, encoder_2, combined_dot])
# Epoch 17/20
# 57512/57512 [==============================] - 12s 205us/step - loss: 0.1726 - acc: 0.9115 - val_loss: 0.3162 - val_acc: 0.8559

# + dropout 0.1: hülft nicht viel... bzw. nichts...

In [23]:
# dev set performance
report_training_results(dev_output, dev_output_pred)

Confusion Matrix:
[[1752  306]
 [ 296 2144]]

Accuracy:  0.87

Report:
             precision    recall  f1-score   support

          0       0.86      0.85      0.85      2058
          1       0.88      0.88      0.88      2440

avg / total       0.87      0.87      0.87      4498



{'macro': 0.87, 'micro': 0.87}

In [24]:
# test set performance
report_training_results(test_output, test_output_pred)

Confusion Matrix:
[[794 114]
 [128 902]]

Accuracy:  0.88

Report:
             precision    recall  f1-score   support

          0       0.86      0.87      0.87       908
          1       0.89      0.88      0.88      1030

avg / total       0.88      0.88      0.88      1938



{'macro': 0.87, 'micro': 0.88}

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         1049600     input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1024)         1049600     input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

## Predict test set probabilities

In [26]:
within_test_all =  pd.read_csv("data/same-side-classification/within-topic/test.csv", index_col='id')

In [27]:
within_test_all = within_test_all.apply(add_tag, axis=1)
within_test_all.tail()

,argument1,argument2,topic,tag
id,,,,
95327,(resolved)on balance:middleclassandrichwomenwh...,(resolved)on balance:middleclassandrichwomenwh...,(resolved)on balance:middleclassandrichwomenwh...,abortion
95345,(resolved)on balance:middleclassandrichwomenwh...,(resolved)on balance:middleclassandrichwomenwh...,(resolved)on balance:middleclassandrichwomenwh...,abortion
95364,gay marriage should be legalized in america i'...,gay marriage should be legalized in america i ...,gay marriage should be legalized in america,gay marriage
95369,"every human being has rights, even if they can...","first, i must say that i do not under any circ...",live birth abortion should stay illegal.,abortion
95370,"yes, but the baby is still not alive, it is st...","until a baby is born naturally, it is not trul...",live birth abortion should stay illegal.,abortion


In [28]:
a1 = bc.encode(within_test_all.argument1.tolist())
a2 = bc.encode(within_test_all.argument2.tolist())
testall_embedded_pairs = zip(a1, a2)
pickle.dump(testall_embedded_pairs, open("BERT_pairs_testall.pkl", "wb"))

NameError: name 'bc' is not defined

In [29]:
testall_embs = pickle.load(open("BERT_pairs_testall.pkl","rb"))
t1, t2 = zip(*testall_embs)
testall_args1 = np.array(t1)
testall_args2 = np.array(t2)
testall_args1.shape
testall_tags = to_categorical([0 if t=='abortion' else 1 for t in within_train_df.tag.tolist()], num_classes=2)

In [30]:
model.load_weights("argmining19_simple-nn-model.hdf5")
testall_output_pred = model.predict([testall_args1, testall_args2, testall_tags])

In [31]:
# write to disk
with open("predicted_labels_simple-nn_within.csv", "w") as f:
    for i, a in enumerate(testall_output_pred):
        f.write(str(within_test_all.index.values[i]) + "," + str(a[0]) + "\n")

In [32]:
testall_output_pred

array([[0.67102396],
       [0.6054134 ],
       [0.9379481 ],
       ...,
       [0.6328175 ],
       [0.63011324],
       [0.02002504]], dtype=float32)

# Cross topic

In [45]:
cross_dev_df = within_train_df[within_train_df.tag == "gay marriage"][:3000]

In [46]:
len(cross_dev_df)

3000

In [47]:
from bert_serving.client import BertClient
bc = BertClient()
# dev set
a1 = bc.encode(cross_dev_df.argument1.tolist())
a2 = bc.encode(cross_dev_df.argument2.tolist())
dev_embedded_pairs = zip(a1, a2)
pickle.dump(dev_embedded_pairs, open("BERT_pairs_cross_dev.pkl", "wb"))

/srv/home/gwiedemann/miniconda3/envs/nnnlp/lib/python3.6/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=256" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [37]:
cross_train_df.head()

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
0,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
1,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
2,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
3,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00002-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00002-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
4,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion


In [36]:
# BERTify training and test data
from bert_serving.client import BertClient
bc = BertClient()
# train set
a1 = bc.encode(cross_train_df.argument1.tolist())
a2 = bc.encode(cross_train_df.argument2.tolist())
train_embedded_pairs = zip(a1, a2)
pickle.dump(train_embedded_pairs, open("BERT_pairs_cross_train.pkl", "wb"))
# dev set
a1 = bc.encode(cross_dev_df.argument1.tolist())
a2 = bc.encode(cross_dev_df.argument2.tolist())
dev_embedded_pairs = zip(a1, a2)
pickle.dump(dev_embedded_pairs, open("BERT_pairs_cross_dev.pkl", "wb"))
# test set
a1 = bc.encode(cross_test_df.argument1.tolist())
a2 = bc.encode(cross_test_df.argument2.tolist())
test_embedded_pairs = zip(a1, a2)
pickle.dump(test_embedded_pairs, open("BERT_pairs_cross_test.pkl", "wb"))

/srv/home/gwiedemann/miniconda3/envs/nnnlp/lib/python3.6/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=256" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [48]:
training_embs = pickle.load(open("BERT_pairs_cross_train.pkl","rb"))
dev_embs = pickle.load(open("BERT_pairs_cross_dev.pkl","rb"))
test_embs = pickle.load(open("BERT_pairs_cross_test.pkl","rb"))

In [49]:
# train set
t1, t2 = zip(*training_embs)
train_args1 = np.array(t1)
train_args2 = np.array(t2)
# dev set
t1, t2 = zip(*dev_embs)
dev_args1 = np.array(t1)
dev_args2 = np.array(t2)
# test set
t1, t2 = zip(*test_embs)
test_args1 = np.array(t1)
test_args2 = np.array(t2)
# outputs
train_output = [1 if t else 0 for t in cross_train_df.is_same_side]
dev_output = [1 if t else 0 for t in cross_dev_df.is_same_side]
test_output = [1 if t else 0 for t in cross_test_df.is_same_side]

In [50]:
# the model
embedding_dims = t1[0].shape

layer_input_1 = Input(shape = embedding_dims, name = 'input_1')
layer_input_2 = Input(shape = embedding_dims, name = 'input_2')

encoder_1 = Dense(embedding_dims[0], activation='relu')(layer_input_1)
encoder_2 = Dense(embedding_dims[0], activation='relu')(layer_input_2)
combined_dot = dot([encoder_1, encoder_2], axes=-1, normalize=False)
combined_all = concatenate([encoder_1, encoder_2, combined_dot])

features_pred = Dense(300, activation='relu')(combined_all)
layer_prediction = Dense(1, activation='sigmoid')(features_pred)

model = Model([layer_input_1, layer_input_2], layer_prediction)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [51]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("argmining19_simple-nn-model_cross.hdf5", monitor='val_acc', 
                             verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
model.fit([train_args1, train_args2], train_output, batch_size=32, epochs=30, 
          validation_data=([dev_args1, dev_args2], dev_output), verbose=True,
          callbacks=[checkpoint])

Train on 54947 samples, validate on 3000 samples
Epoch 1/30
54947/54947 [==============================] - 15s 275us/step - loss: 0.3754 - acc: 0.7774 - val_loss: 1.1004 - val_acc: 0.6043

Epoch 00001: val_acc improved from -inf to 0.60433, saving model to argmining19_simple-nn-model_cross.hdf5
Epoch 2/30
54947/54947 [==============================] - 12s 217us/step - loss: 0.3018 - acc: 0.8125 - val_loss: 1.1684 - val_acc: 0.5833

Epoch 00002: val_acc did not improve from 0.60433
Epoch 3/30
54947/54947 [==============================] - 12s 212us/step - loss: 0.2849 - acc: 0.8248 - val_loss: 1.8151 - val_acc: 0.6057

Epoch 00003: val_acc improved from 0.60433 to 0.60567, saving model to argmining19_simple-nn-model_cross.hdf5
Epoch 4/30
54947/54947 [==============================] - 12s 213us/step - loss: 0.2708 - acc: 0.8344 - val_loss: 1.6992 - val_acc: 0.5933

Epoch 00004: val_acc did not improve from 0.60567
Epoch 5/30
54947/54947 [==============================] - 12s 211us/step -

In [52]:
model.load_weights("argmining19_simple-nn-model_cross.hdf5")
dev_output_pred = np.round(model.predict([dev_args1, dev_args2]))
test_output_pred = np.round(model.predict([test_args1, test_args2]))

In [53]:
# dev set performance
report_training_results(dev_output, dev_output_pred)

Confusion Matrix:
[[ 718  565]
 [ 610 1107]]

Accuracy:  0.61

Report:
             precision    recall  f1-score   support

          0       0.54      0.56      0.55      1283
          1       0.66      0.64      0.65      1717

avg / total       0.61      0.61      0.61      3000



{'macro': 0.6, 'micro': 0.61}

In [54]:
# test set performance
report_training_results(test_output, test_output_pred)

Confusion Matrix:
[[ 935  279]
 [ 139 1088]]

Accuracy:  0.83

Report:
             precision    recall  f1-score   support

          0       0.87      0.77      0.82      1214
          1       0.80      0.89      0.84      1227

avg / total       0.83      0.83      0.83      2441



{'macro': 0.83, 'micro': 0.83}

## Predict test set probabilities

In [58]:
cross_test_all =  pd.read_csv("data/same-side-classification/cross-topic/test.csv", index_col='id')

In [44]:
a1 = bc.encode(cross_test_all.argument1.tolist())
a2 = bc.encode(cross_test_all.argument2.tolist())
testall_embedded_pairs = zip(a1, a2)
pickle.dump(testall_embedded_pairs, open("BERT_pairs_testall_cross.pkl", "wb"))

/srv/home/gwiedemann/miniconda3/envs/nnnlp/lib/python3.6/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=256" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [55]:
testall_embs = pickle.load(open("BERT_pairs_testall_cross.pkl","rb"))
t1, t2 = zip(*testall_embs)
testall_args1 = np.array(t1)
testall_args2 = np.array(t2)
testall_args1.shape

(6163, 1024)

In [56]:
model.load_weights("argmining19_simple-nn-model_cross.hdf5")
testall_output_pred = model.predict([testall_args1, testall_args2])

In [59]:
# write to disk
with open("predicted_labels_simple-nn_cross.csv", "w") as f:
    for i, a in enumerate(testall_output_pred):
        f.write(str(cross_test_all.index.values[i]) + "," + str(a[0]) + "\n")